# Plot netCDF data on a map using matplotlib.basemap

Adapted from http://aosc.umd.edu/~cmartin/python/examples/netcdf_example1.html

For a comprehensive list of matplotlib options, see
Python Data Science Handbook by Jake VanderPlas, Chapter 4. Visualization with Matplotlib
https://www.oreilly.com/library/view/python-data-science/9781491912126/ch04.html

INFO: matplotlib 3.0.1 and basemap are broken  
https://github.com/matplotlib/basemap/issues/435

First we need to import netCDF4-python, Numpy, Matplotlib for plotting, and Basemap for the map

In [ ]:
%matplotlib inline
from netCDF4 import Dataset 
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.basemap import Basemap

Load in the netCDF file

In [ ]:
nc = Dataset('/Users/pag064/DATACUBE-DATA/BoM-AGCD-FTP/precip/data/IDCKZRDAT0_precip_total_r005_20180101_20180101.nc')

Read the variables from the netCDF file and assign them to Python variables

In [ ]:
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
time = nc.variables['time'][:]
v = nc.variables['precip'][:]

Let's create a map centered over Australia

In [ ]:
plt.clf()
#plt.figure(figsize=(8,8))
fig, ax = plt.subplots()
map = Basemap(ax=ax,
              projection='cyl',
              llcrnrlon=110.,
              llcrnrlat=-45.,
              urcrnrlon=155.,
              urcrnrlat=-5.,
              resolution='i')
# resolutions: c - crude, l - low, i - intermediate, h - high, f - full

Now let's add the states, counties, coasts, national borders and a land-sea colored mask:

In [ ]:
map.drawcoastlines()
map.drawstates(a)
map.drawcountries()
map.drawlsmask(land_color='linen', ocean_color='#CCFFFF') # can use HTML names or codes for colors
map.drawcounties() # you can even add counties (and other shapefiles!)

If you'd like to add Lat/Lon, you can do that too:

In [ ]:
parallels = np.arange(-45,-5,5.) # make latitude lines ever 5 degrees from 30N-50N
meridians = np.arange(110,155,5.) # make longitude lines every 5 degrees from 95W to 70W
map.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
map.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
plt.show()

Now, let's prepare the data for the map.
We have to transform the lat/lon data to map coordinates.

In [ ]:
lons,lats= np.meshgrid(lon,lat)
x,y = map(lons,lats)

After that, we can then plot the netCDF data on the map.

In [ ]:
print(v.min(), v.max(), v.shape)
print(x.min(), x.max(), x.shape)
print(y.min(), y.max(), y.shape)
clevs = np.arange(10,100,10)
print(clevs)
cs = map.contour(x,y,v[0,:,:],clevs,colors='blue',linewidths=1.)

Finally, let's add a title.

In [ ]:
plt.clabel(cs, fontsize=9, inline=1) # contour labels
plt.title('24 hour rainfall')
plt.show()

If you'd rather plot precipitation, we can do that too (and with a colorbar).

In [ ]:
data = map.contourf(x,y,v[0,:,:])
cb = plt.colorbar(data)
cb.set_label('Rainfall (mm)')